## Solution: Setting up the Agents

## Install the packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [1]:
import os, jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    BingGroundingTool, 
    MessageRole, 
    FileSearchTool, 
    FilePurpose,
    OpenApiTool, 
    OpenApiConnectionAuthDetails, 
    OpenApiConnectionSecurityScheme
) 

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Run Agent Helper Function


In [4]:
def run_agent(user_input, thread_id, agent_id):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread_id,  # ID of the thread to which the message belongs
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread_id, agent_id=agent_id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread_id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Create Weather Agent

In [7]:
# Enter Bing connection names here
bing_search = "bingresource"  # Replace with your Bing connection name

# Extract the connection list.
conn_list = project_client.connections.list()
bing_search_conn_id = ""

# Find the connection IDs for Bing Search and Bing Custom Search
for conn in conn_list:
    if conn.name == bing_search:
        bing_search_conn_id = conn.id

# Print the connection names
print(f"Bing Search Connection ID: {bing_search_conn_id}")

# Initialize the Bing Grounding tools
bing_search = BingGroundingTool(connection_id=bing_search_conn_id)

# Create an agent with the Bing Grounding tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="weather-agent",  # Name of the agent
    instructions="You are a helpful agent that provides weather information",  # Instructions for the agent
    tools=bing_search.definitions  # Attach the Bing Grounding tool
)

print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Example user input for the agent
user_input = "What is the weather in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input


Bing Search Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent_challenge/providers/Microsoft.CognitiveServices/accounts/challengeptff/projects/contoso_projectptff/connections/bingresource
Created agent, ID: asst_TWn4ilX0ZBWbcRIIhsQI4jF9
Created thread, ID: thread_nSaCT8ZswRfydlZvdU1nCRsR
Created message, ID: msg_MLDpU8ZrStHcunwMetojn379
Run finished with status: completed
Agent Response: The current weather in Paris is relatively mild, with temperatures around 18°C early in the morning and expected to rise to a high of 29°C during the day. There is no significant chance of rain, and the conditions are clear with plenty of sunshine. Winds are light, blowing from the northeast at around 9 km/h, and humidity levels are around 51%【3:0†source】【3:2†source】【3:3†source】.


## Create Itinerary Agent

In [8]:
# Create an agent with the Bing Grounding tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="itinerary-agent",  # Name of the agent
    instructions="You are a helpful agent that creates an itinerary given the cities' weather information.",  # Instructions for the agent
)

print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

user_input = """Create an itinerary for Paris given this weather information:
Currently, in Paris, it is partly cloudy with a temperature of 68°F (20°C). 
The forecast for later today predicts heavy downpours and frequent lightning with afternoon thundershowers. 
The high temperature will reach 69°F (21°C), with a 100% chance of rain
"""  # Example user input

run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created agent, ID: asst_sKhtRu2UTeBVSlEjJvXahEOz
Created thread, ID: thread_BnO7Y9QW1W84JLGZZmbBaCSw
Created message, ID: msg_k13gGxTe1Vl5kFUES8gMvylL
Run finished with status: completed
Agent Response: Sure, I'll create a day itinerary for Paris considering the current weather conditions and the forecast for later today. 

### Morning
**8:00 AM - 10:00 AM:**
- **Breakfast at a Café:** Start your day with a classic Parisian breakfast at a local café. Enjoy fresh croissants, baguettes, and a café au lait.

**10:00 AM - 12:00 PM:**
- **Visit the Eiffel Tower:** Since it's currently partly cloudy, the morning is the best time to visit the Eiffel Tower. You can enjoy the view from both the ground and the top, but be prepared for some potential crowding.

### Midday
**12:00 PM - 1:00 PM:**
- **Lunch at Le Jules Verne:** Dine at the renowned restaurant located on the Eiffel Tower. Enjoy a superb meal while taking in the view of Paris.

### Early Afternoon
**1:00 PM - 3:00 PM:**
- **Louvre Mu

## Travel Product Agent

In [9]:
file_directory = "../data/travelproducts"  # Directory containing files to upload

# List all files in the directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

# Get the full path of a file
def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)

# Upload files to the project client
file_ids: list[str] = []
for path in [get_filepath_for_filename(filename) for filename in filenames]:
    with open(path, "rb") as file:
        file = project_client.agents.files.upload_and_poll(file_path=path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file, file ID: {file.id}")
        file_ids.append(file.id)

# Create a vector store with the uploaded files
vector_store = project_client.agents.vector_stores.create_and_poll(file_ids=file_ids, name="travelproducts_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

# Create a file search tool
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="travelproducts-agent",  # Name of the agent
    instructions="You are a helpful agent and that will provide travel products information from uploaded files",  # Instructions for the agent
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Example user input for the agent
user_input = "What are the features of the UrbanTraveler?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  #

['SafeJourney Travel Insurance.docx', 'UrbanTech Laptop Messenger Bag.docx', 'Urban Explorer Daypack.docx', 'Adventure Seeker Sling.docx', 'JetSet Travel Pillow.docx', 'TravelBuddy Digital Luggage Scale.docx', 'TravelConnect WiFi Plan.docx', 'WanderSafe Travel Lock.docx', 'Nomad Traveler Suitcase.docx', 'TechGuard Laptop Bag.docx', 'Metro Commuter Crossbody Bag.docx', 'UrbanTraveler Laptop Backpack.docx', 'Voyager Pro Luggage.docx', 'Executive Pro Laptop Briefcase.docx', 'TravelEase Packing Cubes Set.docx', 'TravelSmart Universal Adapter.docx', 'Globetrotter Elite Suitcase.docx', 'AdventurePro Hiking Backpack.docx']
Uploaded file, file ID: assistant-1M4ya6ssfNwHfbgdeoqwb9
Uploaded file, file ID: assistant-4d25vXyWsq8pAUkJ24A5df
Uploaded file, file ID: assistant-NAxZUjL24LBjX1hVq538Qd
Uploaded file, file ID: assistant-1C1UNVvHNoKBZSpaZTSHS7
Uploaded file, file ID: assistant-AqdVLyxa7kFSN4RwgbQpYQ
Uploaded file, file ID: assistant-BgXMNJ3K2v2ejR5EHuWSGZ
Uploaded file, file ID: assistant-

## Restaurant Agent

In [11]:
# Enter TripAdvisor connection names here
tripadvisor = "tripadvisor"  # Replace with your TripAdvisor connection name

# Load the OpenAPI specification from a JSON file
with open('../data/tripadvisor/tripadvisor_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Extract the connection list.
conn_list = project_client.connections.list()
tripadvisor_conn_id = ""

# Find the connection IDs for TripAdvisor
for conn in conn_list:
    if conn.name == tripadvisor:
        tripadvisor_conn_id = conn.id
  
# Print the connection names
print(f"TripAdvisor Connection ID: {tripadvisor_conn_id}")

# Create the OpenAPI tool
auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=tripadvisor_conn_id))
tripadvisor_tool = OpenApiTool(
                    name=tripadvisor, 
                    spec=openapi_spec, 
                    description="get hotel and restaurant reviews of a location", 
                    auth=auth
                )

# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="restaurant-agent",  # Name of the agent
    instructions="You are a helpful agent that provides restaurant recommendations and reviews using TripAdvisor",  # Instructions for the agent
    tools=tripadvisor_tool.definitions,  # Tools available to the agent
    tool_resources=tripadvisor_tool.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Example user input for the agent
user_input = """
    Provide me 3 reviews of the Pizzeria Arrivederci in Paris.    
"""  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  

TripAdvisor Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent_challenge/providers/Microsoft.CognitiveServices/accounts/challengeptff/projects/contoso_projectptff/connections/tripadvisor
Created agent, ID: asst_DL2iuQSceBrvKIPV9G9kW0mF
Created thread, ID: thread_hUmSRBhfAiRLSUIYvvL2MvAd
Created message, ID: msg_jKIA3E4Zh8nmfXEKT5aANATp
Run finished with status: completed
Agent Response: Here are 3 reviews of the Pizzeria Arrivederci in Paris:

1. **Review by sannak926**:
   - **Rating**: ★★★★★
   - **Title**: Nice and cozy little pizzeria
   - **Date**: July 13, 2025
   - **Review**: Very good menu and very delicious pizzas. Very popular place! Our table booking was right on time and staff was very friendly!
   - **Trip Type**: Couples
   - [Read full review](https://www.tripadvisor.com/ShowUserReviews-g187147-d12714552-r1018027105-Reviews-Pizzeria_Arrivederci-Paris_Ile_de_France.html?m=66827#review1018027105)

2. **Review by F7796YTingridt**:
   - 